## Sender Simulating Loss:



In [ ]:
import os
from socket import *
import time
import random
from datetime import datetime
import matplotlib.pyplot as plt



def plot_packet_times(sent_packets, acknowledged_packets, packets_trans_count, loss_rate, num_retrans, timeout):
    ack_packet_ids, ack_times = zip(*acknowledged_packets)

    single_sent_packets = [i for i in ack_packet_ids if packets_trans_count[i] == 1]
    single_sent_times = [ack_times[i] for i in range(len(ack_packet_ids)) if
                         packets_trans_count[ack_packet_ids[i]] == 1]

    retransmitted_packets = [i for i in ack_packet_ids if packets_trans_count[i] > 1]
    retransmitted_times = [ack_times[i] for i in range(len(ack_packet_ids)) if
                           packets_trans_count[ack_packet_ids[i]] > 1]

    plt.scatter(single_sent_times, single_sent_packets, color='blue', label='Sent once')
    plt.scatter(retransmitted_times, retransmitted_packets, color='red', label='Retransmitted')

    plt.xlabel('Time taken (seconds)')
    plt.ylabel('Packet ID')
    plt.title('Packet ID vs Time taken for acknowledgment')

    # Add text annotations to display Loss rate, Number of Retransmissions, and Timeout interval
    text_str = f'Loss rate: {"{:.2f}".format(loss_rate)}\nNumber of Retranssmissions: {num_retrans}\nTimeout interval: {timeout} sec'
    plt.gca().text(0.95, 0.05, text_str, transform=plt.gca().transAxes, fontsize=10, verticalalignment='bottom',
                   horizontalalignment='right', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))

    plt.legend()
    plt.show()


def str_to_int(bits_str):
    int_val = int(bits_str, 2)
    return int_val


def Dividing_file_into_packets(file_path, MSS, file_id):
    file_size = os.path.getsize(file_path)
    packets = []

    with open(file_path, "rb") as file:  # read file in binary mode

        file_data = file.read()
        num_packets = file_size // MSS + 1

    for j in range(num_packets):

        start_index = j * MSS
        end_index = start_index + MSS

        # Slice the file data to get the current chunk
        chunk_data = file_data[start_index:end_index]
        # construct packet
        packet_ID = f'{j :016b}'
        file_ID = file_id
        if (j == num_packets - 1):
            hex_num = 0xFFFFFFFF
            Trailer = bin(hex_num)[2:]
        else:
            Trailer = f'{0:032b}'
        packet = str(packet_ID) + str(file_ID) + str(chunk_data) + str(Trailer)
        packets.append(packet)

    return packets


def send(file_pa0th, receiver_IP_address, receiver_port):
    sent_packets = []
    acknowledged_packets = []
    serverName = receiver_IP_address
    serverPort = receiver_port
    clientSocket = socket(AF_INET, SOCK_DGRAM)
    timeout = 1
    clientSocket.settimeout(timeout)
    cwnd = 1
    window_base = 0
    ssthresh = 8
    num_retrans = 0  # counting number of retransmissions
    last_sent_packet = -1
    total_sent_packets = 0
    MSS = 1024-64# Maximum segment size
    file_id = 1000000000000000
    array_of_packets = Dividing_file_into_packets(file_path, MSS, file_id)
    packets_trans_count = {}
    num_packets = len(array_of_packets)
    print("Total num of packets", num_packets)

    last_ack_rec = -1
    ACK_NUM = -1

    received_packets = 0  # for each sending window
    ## Regarding Start Time
    st = datetime.now()
    Transfer_start_time = st.strftime('%H:%M:%S')
    start_sec = time.time()

    while ACK_NUM + 1 < num_packets:

        if last_sent_packet + 1 < num_packets:
            # start with cwnd = 1 (slow start) in the begining of the file or right after a loss/time out are detected
            clientSocket.sendto(array_of_packets[last_sent_packet + 1].encode(), (serverName, serverPort))
            print(" I have sent ", last_sent_packet + 1)
            sent_packets.append((last_sent_packet + 1, time.time()))  # After sending a packet
            if packets_trans_count.get(last_sent_packet + 1) == None:
                packets_trans_count[last_sent_packet + 1] = 0
            else:
                packets_trans_count[last_sent_packet + 1] += 1
            last_sent_packet = last_sent_packet + 1

        try:
            # continue sending a packet at accelerating rate as long as there is no timeout error nor loss nor the max acks was reached
            while ACK_NUM <= window_base + cwnd:
                Ack, address = clientSocket.recvfrom(MSS)
                ACK_NUM = str_to_int(Ack.decode()[0:16])

                # case of no loss
                # exponential increase in the cwnd (slow start)
                if cwnd >= 32:
                   cwnd = 32
                elif cwnd < ssthresh:
                    # slow start as long as ssthresh is not reached yet
                    cwnd = min(2 * cwnd, ssthresh)
                else:  # linear increase (congestion avoidance)
                    cwnd = cwnd + 1

                # for the purpose of reseting the cwnd when loss/time out is detected
                if (ACK_NUM >= last_ack_rec):
                    received_packets = ACK_NUM - last_ack_rec
                    last_ack_rec = ACK_NUM

                    acknowledged_packets.append((ACK_NUM, time.time()))  # After Ack
                    print("Packet", last_ack_rec, "acknowledged")
                    window_base = ACK_NUM + 1
                    packets_to_sent = received_packets
                    print("Window base updated to:", window_base)
                    # as long as there are yet backets to send in the cwnd

                    for j in range(packets_to_sent):
                        # for if there is still an empty space in the cwnd despite the fact that this was the final number of packets and the cwnd is greater than that number.
                        if last_sent_packet + 1 < num_packets:
                            num_retrans = 0
                            # send as long as the cwnd contains a free space and the number of Packets supposed to be sent, is not reached yet.
                            for key in packets_trans_count:
                                value = packets_trans_count[key]
                                num_retrans = num_retrans + value
                            loss_rate = (num_retrans / num_packets) * 100
                            rand = random.uniform(0, 10)
                            if rand > 4 or loss_rate > 20:
                                clientSocket.sendto(array_of_packets[last_sent_packet + 1].encode(),
                                                    (serverName, serverPort))
                                # set the new time out right after sending
                                sent_packets.append((last_sent_packet + 1, time.time()))  # After sending a packet

                                print("Last sent", last_sent_packet + 1)
                                if packets_trans_count.get(last_sent_packet + 1) == None:
                                    packets_trans_count[last_sent_packet + 1] = 0
                                else:
                                    packets_trans_count[last_sent_packet + 1] += 1
                                # increment the sent packets by one to send the upcoming backet by next iteration
                                last_sent_packet +=1
                            else:
                                if loss_rate < 20:
                                   # last_sent_packet  +=1
                                    print("Loss is occurred")
                                    continue
                                else:
                                    clientSocket.sendto(array_of_packets[last_sent_packet + 1].encode(),
                                                        (serverName, serverPort))
                                    # set the new time out right after sending
                                    sent_packets.append((last_sent_packet + 1, time.time()))  # After sending a packet

                                    print("Last sent from else", last_sent_packet + 1)
                                    if packets_trans_count.get(last_sent_packet + 1) == None:
                                        packets_trans_count[last_sent_packet + 1] = 0
                                    else:
                                        packets_trans_count[last_sent_packet + 1] += 1
                                    last_sent_packet += 1




        except OSError:
            if last_ack_rec == num_packets - 1:
                print("Last Ack", last_ack_rec)
                print("All packets sent successfully")
                break
            print("Timeout occurred, resending packets starting from:", window_base)
            # setting the cwnd and the ssthresh right after the loss/time out
            ssthresh = cwnd / 2
            cwnd = 1
            if last_ack_rec != -1:
                last_sent_packet = ACK_NUM
            continue

    # End Time
    en = datetime.now()
    Transfer_END_time = en.strftime('%H:%M:%S')
    Transfer_end_time = time.time()
    elapsed_time = en - st
    elapsed_time_sec = Transfer_end_time - start_sec

    ''' Statistics after Trasmitting each File'''

    num_of_bytes = os.path.getsize(file_path)
    # number of retransmissions
    num_retrans = 0
    for key in packets_trans_count:
        value = packets_trans_count[key]
        num_retrans = num_retrans + value

    loss_rate = (num_retrans / num_packets) * 100
    Average_trans_rate_bytes = num_of_bytes / elapsed_time_sec
    Average_trans_rate_packets = num_packets / elapsed_time_sec

    # General Statistics:

    print("Transfer start time = ", Transfer_start_time, "sec")
    print("Transfer end time   = ", Transfer_END_time, "sec")
    print("Elapsed Time        = ", elapsed_time, "sec")
    print("Number of Packets   = ", num_packets, "packet")
    print("Number of Bytes     = ", num_of_bytes, "byte")
    print("Number of Retransmissions  = ", num_retrans)
    print("Average transfer rate   = ", "{:.2f}".format(Average_trans_rate_bytes), 'bytes/sec')
    print("Average transfer rate  = ", "{:.2f}".format(Average_trans_rate_packets), 'packets/sec')
    print("loss rate  = ", "{:.2f}".format(loss_rate), '%')

    return sent_packets, acknowledged_packets, packets_trans_count, loss_rate, num_retrans, timeout

file_path = r''
receiver_port = 12000
receiver_IP_address = ''
sent_packets, acknowledged_packets,packets_trans_count,loss_rate, num_retrans, timeout=send(file_path, receiver_IP_address, receiver_port)
plot_packet_times(sent_packets, acknowledged_packets, packets_trans_count,loss_rate, num_retrans, timeout) # Call this function after the send() function to plot the results




## Receiver:

In [ ]:
import socket
import ast

IP_Address = ''
PORT = 12000
Socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
Socket.bind(("", PORT))
MSS = 1024
trailer_size = 4

import struct

def unpack_packet(packet):
    """
    Unpacks the packet and returns the packet ID, file ID, application data, and trailer value.
    """
    packetIDSize = 16  # 16 bits packetID
    fileIDSize = 16  # 16 bits fileID
    trailerSize = 32 # 16 bits trailer
    
    packet_id = packet[:packetIDSize] # First 16 bits
    file_id = packet[fileIDSize:fileIDSize+packetIDSize] # Second 16 bits
    application_data = packet[trailerSize:-(fileIDSize+packetIDSize)] # Data
    trailer = packet[-trailerSize:] # last 32 bits

    return packet_id, file_id, application_data, trailer

def pack_ack_packet(packet_id, file_id):
    """
    Packs the given packet ID and file ID into an acknowledgment packet.
    """
    
   
    packet_ack = str(packet_id) + file_id
    
    return packet_ack



def receiver(Socket,MSS,Trailer_Size):

  """
  1- Bind the UDP socket to the IP Adress and the port number
  2- Create a loop to keep reciveing the packets from the sender
  3- Parse the received packet using unpack_packet function (still needs to be implemented)
  4- Check if the packet_id is the same as the expected_packet_id if so then store the data in the data buffer and increment the excpected packet id
  5- Send an ACK to the sender with the last received correct packet
  6- check if the trailer value is 0xFFFF if so then the last packet of that file has been received

  """

# Initialize variables
  expected_packet_id = 0
  file_id = -1 #Constant
  data_buffer = {}

  while True: #Receive data until the end of the file
     packet, addr = Socket.recvfrom(MSS + Trailer_Size) #Get the next packet
     packet_id, file_id, data, trailer = unpack_packet(packet.decode())
     packet_id_decimal = int(packet_id, 2)
     print("I received packet",str(packet_id_decimal))
     """
     The unpack_packet function still needs to be implemented
     """
  
     
     if packet_id_decimal == expected_packet_id and file_id != -1: # Check if the packet is the expected one
        data_buffer[packet_id_decimal] = data # Add the packet data to the buffer

        # Check if data_buffer size exceeds 10 MiB (10485760 bytes)   Protocol 2
        data_buffer_size = sum(sys.getsizeof(v) for v in data_buffer.values())
        if data_buffer_size >= 10485760:
            print("The connection is aborted due to exceeding the maximum data buffer size.")
            break
      
        # Send an ACK packet to the sender
       
        ack_packet = pack_ack_packet(packet_id, file_id)
        print('ack',ack_packet)
        
        Socket.sendto(ack_packet.encode(), addr)
        expected_packet_id += 1 # Update the expected packet ID for the next packet
 
      
          
        if trailer == str(11111111111111111111111111111111):
         
          new_file_path = r'D:\HELLO.png'
          bytes_buffer = {packet_id: ast.literal_eval(packet_data) for packet_id, packet_data in data_buffer.items()}
          combined_bytes = b''.join(bytes_buffer.values())
          with open(new_file_path, "wb") as f:  # write file in binary mode
            f.write(combined_bytes)
            print(f'File {file_id} received.')

            """
            A function needs to be implemented to deal with colored images (transfer colored images data buffer into image.png)
            """

            # Reset the buffer and expected packet ID for the next file
            data_buffer = {}
            expected_packet_id = 0
            file_id = -1

            # Ask the user if they want to receive another file
            user_input = input("Do you want to receive another file? (yes/no) ")
            if user_input.lower() != "yes":
                break

        
            
    #  else: # Discard the packet and send an ACK for the last correctly received packet
    #     ack_packet = pack_ack_packet(expected_packet_id-1, file_id)
    #     Socket.sendto(ack_packet.encode(), addr)

  

receiver(Socket,MSS,trailer_size)